In [1]:
#pers
filter=[[21888, u'en', u'Introduction to Psychology'], [26707, u'en', u'SQL Server - Real World Troubleshooting Skills - Udemy'], [16487, u'es', u'Aprende c\xf3mo crear gr\xe1ficos en 3D con 3Ds Max y Vray'], [23487, u'es', u'Esquemas, subtotales y an\xe1lisis de datos con Excel'], [12201, u'ru', u'\u0412\u0432\u0435\u0434\u0435\u043d\u0438\u0435 \u0432 \u0440\u0430\u0437\u0440\u0430\u0431\u043e\u0442\u043a\u0443 \u043f\u0440\u0438\u043b\u043e\u0436\u0435\u043d\u0438\u0439 \u0434\u043b\u044f \u041e\u0421 Android'], [982, u'ru', u'Flash MX Studio']]



## Запускаем PySpark

In [2]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
sc.getConf().getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.maxExecutors', '14'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.history.fs.cleaner.interval', '7d'),
 ('spark.shuffle.io.serverThreads', '128'),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.executorEnv.PYTHONPATH',
  '{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip'),
 ('spark.shuffle.file.buffer', '1m'),
 ('spark.sql.hive.convertMetastoreOrc', 'true'),
 ('spark.yarn.historyServer.address', 'master.cluster-lab.com:18081'),
 ('spark.sql.autoBroadcastJoinThreshold', '26214400'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),


## Импортируем библиотеки

In [4]:
import numpy as np
import pandas as pd
import os
import json,codecs
import math
from pyspark.sql import functions as F

from pyspark.sql.functions import col, expr, when, desc,sum,count,udf,pandas_udf,PandasUDFType
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.sql.types import *


# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")


## Загружаем данные

In [5]:
schema = StructType([StructField(str(i), tp, True) for i,tp in zip(('id','lang','name'),(IntegerType(),StringType(),StringType()))])

mydata=spark.createDataFrame(filter, schema)
mydata.toPandas()

,id,lang,name
0,21888,en,Introduction to Psychology
1,26707,en,SQL Server - Real World Troubleshooting Skills...
2,16487,es,Aprende cómo crear gráficos en 3D con 3Ds Max ...
3,23487,es,"Esquemas, subtotales y análisis de datos con E..."
4,12201,ru,Введение в разработку приложений для ОС Android
5,982,ru,Flash MX Studio


In [6]:
lab_data = '/labs/lab07data/DO_record_per_line.json'

data = spark.read.json(lab_data)


In [7]:
data.limit(5).toPandas()

,cat,desc,id,lang,name,provider
0,5/computer_science,Shows how to share content quickly and easily ...,16308,en,Up and Running with Tumblr,Lynda
1,3/business_management,Learn how to get up and running on Twitter and...,16309,en,Up and Running with Twitter,Lynda
2,5/computer_science,Learn how to use Typekit to choose from thousa...,16310,en,Up and Running with Typekit,Lynda
3,5/computer_science,Simplify your code and eliminate repetitive da...,16311,en,Up and Running with Underscore.js,Lynda
4,3/business_management|5/computer_science,Introduces object-oriented programming and pro...,16312,en,Up and Running with VBA in Access,Lynda


In [8]:
data.createOrReplaceTempView("data")
mydata.createOrReplaceTempView("mydata")

data_sort = spark.sql('select IF(md.id is null,0,1) as my,d.* from mydata md right outer join data d on md.id=d.id order by IF(md.id is null,0,1) desc ')
#ratings.createOrReplaceTempView("ratings")
data_sort.limit(7).toPandas()

,my,cat,desc,id,lang,name,provider
0,1,9/humanities,"This course is an introduction to psychology, ...",21888,en,Introduction to Psychology,edX
1,1,,Watch as I use one free tool to diagnose SQL ...,26707,en,SQL Server - Real World Troubleshooting Skills...,Udemy
2,1,1/arts_music_film,"\nEn este curso realizaremos el modelado, text...",16487,es,Aprende cómo crear gráficos en 3D con 3Ds Max ...,Udemy
3,1,5/computer_science,"Курс для тех, кто профессионально работает с F...",982,ru,Flash MX Studio,Intuit
4,1,,Aprende a utilizar herramientas de análisis d...,23487,es,"Esquemas, subtotales y análisis de datos con E...",Udemy
5,1,5/computer_science,"Курс ориентирован на людей, имеющих небольшой ...",12201,ru,Введение в разработку приложений для ОС Android,Intuit
6,0,14/social_sciences,What is American foreign policy? Who makes it?...,65,en,21st Century American Foreign Policy,Coursera


## Подготовка данных

In [9]:
from pyspark.sql.types import ArrayType
import re

regex = re.compile('[\w\d]{2,}', re.U)

spark.udf.register("split", lambda line: regex.findall(line.lower()))
split = udf(lambda line: regex.findall(line.lower()), ArrayType(StringType()))

In [10]:
data_split=data_sort.withColumn("desc_spl", split(data_sort.desc))
data_split.limit(7).toPandas()

,my,cat,desc,id,lang,name,provider,desc_spl
0,1,5/computer_science,"Курс для тех, кто профессионально работает с F...",982,ru,Flash MX Studio,Intuit,"[курс, для, тех, кто, профессионально, работае..."
1,1,5/computer_science,"Курс ориентирован на людей, имеющих небольшой ...",12201,ru,Введение в разработку приложений для ОС Android,Intuit,"[курс, ориентирован, на, людей, имеющих, небол..."
2,1,9/humanities,"This course is an introduction to psychology, ...",21888,en,Introduction to Psychology,edX,"[this, course, is, an, introduction, to, psych..."
3,1,,Aprende a utilizar herramientas de análisis d...,23487,es,"Esquemas, subtotales y análisis de datos con E...",Udemy,"[aprende, utilizar, herramientas, de, análisis..."
4,1,,Watch as I use one free tool to diagnose SQL ...,26707,en,SQL Server - Real World Troubleshooting Skills...,Udemy,"[watch, as, use, one, free, tool, to, diagnose..."
5,1,1/arts_music_film,"\nEn este curso realizaremos el modelado, text...",16487,es,Aprende cómo crear gráficos en 3D con 3Ds Max ...,Udemy,"[en, este, curso, realizaremos, el, modelado, ..."
6,0,3/business_management,Effective communication skills are recognized ...,26,en,Introduction to Business Communication,Canvas Network,"[effective, communication, skills, are, recogn..."


## Считаем вектора

In [11]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(numFeatures=10000,inputCol="desc_spl", outputCol="tf")
tf = hashingTF.transform(data_split)

idf = IDF(inputCol="tf", outputCol="tfidf")
tfidf = idf.fit(tf).transform(tf)
tfidf.limit(7).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2102: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,my,cat,desc,id,lang,name,provider,desc_spl,tf,tfidf
0,1,9/humanities,"This course is an introduction to psychology, ...",21888,en,Introduction to Psychology,edX,"[this, course, is, an, introduction, to, psych...","(0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 5.8872447544999575, 0.0, 0.0, 0.0, 0.0, ..."
1,1,,Watch as I use one free tool to diagnose SQL ...,26707,en,SQL Server - Real World Troubleshooting Skills...,Udemy,"[watch, as, use, one, free, tool, to, diagnose...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,1/arts_music_film,"\nEn este curso realizaremos el modelado, text...",16487,es,Aprende cómo crear gráficos en 3D con 3Ds Max ...,Udemy,"[en, este, curso, realizaremos, el, modelado, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,5/computer_science,"Курс для тех, кто профессионально работает с F...",982,ru,Flash MX Studio,Intuit,"[курс, для, тех, кто, профессионально, работае...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,,Aprende a utilizar herramientas de análisis d...,23487,es,"Esquemas, subtotales y análisis de datos con E...",Udemy,"[aprende, utilizar, herramientas, de, análisis...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,5/computer_science,"Курс ориентирован на людей, имеющих небольшой ...",12201,ru,Введение в разработку приложений для ОС Android,Intuit,"[курс, ориентирован, на, людей, имеющих, небол...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,0,9/humanities,This first installment of a four-part series i...,19,en,First Peoples to the Early Republic: Born in C...,Canvas Network,"[this, first, installment, of, four, part, ser...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## Нормализуем

In [12]:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="tfidf", outputCol="tfidf_norm")
tfidf_norm = normalizer.transform(tfidf)
tfidf_norm.limit(7).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2102: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,my,cat,desc,id,lang,name,provider,desc_spl,tf,tfidf,tfidf_norm
0,1,5/computer_science,"Курс для тех, кто профессионально работает с F...",982,ru,Flash MX Studio,Intuit,"[курс, для, тех, кто, профессионально, работае...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,,Aprende a utilizar herramientas de análisis d...,23487,es,"Esquemas, subtotales y análisis de datos con E...",Udemy,"[aprende, utilizar, herramientas, de, análisis...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,9/humanities,"This course is an introduction to psychology, ...",21888,en,Introduction to Psychology,edX,"[this, course, is, an, introduction, to, psych...","(0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 5.8872447544999575, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.12015305984339936, 0.0, 0.0, 0.0, 0.0,..."
3,1,5/computer_science,"Курс ориентирован на людей, имеющих небольшой ...",12201,ru,Введение в разработку приложений для ОС Android,Intuit,"[курс, ориентирован, на, людей, имеющих, небол...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,,Watch as I use one free tool to diagnose SQL ...,26707,en,SQL Server - Real World Troubleshooting Skills...,Udemy,"[watch, as, use, one, free, tool, to, diagnose...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,1/arts_music_film,"\nEn este curso realizaremos el modelado, text...",16487,es,Aprende cómo crear gráficos en 3D con 3Ds Max ...,Udemy,"[en, este, curso, realizaremos, el, modelado, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,0,11/law,Taught by one of the world’s leading experts i...,281,en,Introduction to International Criminal Law,Coursera,"[taught, by, one, of, the, world, leading, exp...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## Посчитаем похожесть векторов используя сортировку по перемножению векторов и оконную функцию

In [35]:
dot_udf = udf(lambda x,y: float(x.dot(y)), DoubleType())
spark.udf.register("dot_udf",dot_udf)

tfidf_norm.createOrReplaceTempView("tfidf_norm")

result = spark.sql('with my as (\
                            select id,tfidf_norm,lang,name\
                            from tfidf_norm\
                            where tfidf_norm.my = 1),\
                          other as (\
                             select id,tfidf_norm,lang,name\
                              from tfidf_norm\
                              where tfidf_norm.my = 0),\
                          distances as (\
                             select m.id as my_id, d.id other_id,d.name as other_name,dot_udf(m.tfidf_norm,d.tfidf_norm) similarity \
                             from my m inner join other d on m.lang=d.lang),\
                          similars as (\
                              select distances.*,row_number() OVER (PARTITION BY my_id ORDER BY similarity desc) as rn\
                              from distances)\
                          select *\
                          from similars\
                          where rn<=10\
                          order by my_id,similarity desc,other_name,other_id\
                        ').coalesce(1)
#ratings.createOrReplaceTempView("ratings")
result.toPandas()



,my_id,other_id,other_name,similarity,rn
0,982,26327,Понимаем Lightroom,0.289188,1
1,982,888,Основы работы с CorelDRAW X3,0.277452,2
2,982,858,Методы экспертных оценок,0.230257,3
3,982,25827,Глянцевые Волосы Мгновенно! - Udemy,0.218227,4
4,982,1173,Microsoft Windows для начинающего пользователя,0.217669,5
5,982,8756,Дополнительные вступительные испытания по мате...,0.216422,6
6,982,1175,Microsoft Word для начинающего пользователя,0.214409,7
7,982,8192,Разработка Windows Store приложений на HTML/Ja...,0.208625,8
8,982,8191,Разработка Windows Store приложений на XAML/C#,0.208625,9
9,982,13701,Введение в микроэкономику,0.201165,10


## Сохраним результат

In [36]:
result.filter(result.my_id == 26707 ).select('other_id').collect()

[Row(other_id=7057),
 Row(other_id=19263),
 Row(other_id=4722),
 Row(other_id=26376),
 Row(other_id=6386),
 Row(other_id=19368),
 Row(other_id=17363),
 Row(other_id=18951),
 Row(other_id=24487),
 Row(other_id=6542)]

In [37]:
output = dict()
for i in filter:
    output[i[0]]=list(result.filter(result.my_id == i[0] ).select('other_id').rdd.flatMap(lambda x: x).collect())
output

{21888: [431, 4477, 5986, 7520, 23520, 343, 24864, 15192, 6678, 5631],
 26707: [7057, 19263, 4722, 26376, 6386, 19368, 17363, 18951, 24487, 6542],
 16487: [12453, 12339, 12303, 11634, 12243, 12298, 12338, 11580, 12336, 12337],
 23487: [22681, 23506, 25383, 3902, 18813, 12827, 25902, 11575, 10034, 12461],
 12201: [12725, 13057, 1374, 1365, 1001, 18331, 1247, 1344, 20290, 1332],
 982: [26327, 888, 858, 25827, 1173, 8756, 1175, 8192, 8191, 860]}

## Сохраняем сразу для обоих чекеров

In [31]:
with open('/data/home/sergey.antonov/lab07.json','w',encoding='UTF-8') as data_file:    
     json.dump(output, data_file)

In [18]:
with open('/data/home/sergey.antonov/lab07s.json','w',encoding='UTF-8') as data_file:    
     json.dump(output, data_file)

In [27]:


sc.stop()